In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import selenium
import json
import requests
import re
import time
import Parser3000

# Step1

In [2]:
authors = {
  "Дарья Донцова":  29369,
  "Джеймс Роллинс": 29442,
  "Макс Фрай":      102994,
  "Эрин Хантер":    26149,
  "Дмитрий Емец":   35952
}

In [3]:
def get_authors_id(id_):
    array_id = []
    with webdriver.Firefox() as driver:
        driver.get("https://www.bookvoed.ru/author/books?id="+str(id_))
        last_height = driver.execute_script("return document.body.scrollHeight")
        flag = 0
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1)
            new_height = driver.execute_script("return document.body.scrollHeight")
            time.sleep(1)
            if new_height == last_height:
                time.sleep(1)
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(3)
                try:
                    ty_ = driver.find_element_by_class_name("ty")
                    n = driver.find_element_by_class_name("ty")
                    n.click()
                    time.sleep(1)
                    flag = 1
                except NoSuchElementException:
                    flag = 0
                if(flag == 0):
                    break
            last_height = new_height
        items = driver.find_elements_by_class_name('gf')
        for item in items:
            array_id.append(item.get_attribute('data-book'))
    return array_id

In [4]:
auth_book_id = {}
for i in authors:
    auth_book_id[i] = get_authors_id(authors[i])

# Step 2

In [6]:
all_id = []
for i in auth_book_id:
    for j in auth_book_id[i]:
        all_id.append(int(j))

In [9]:
from multiprocessing import Pool, Lock, Value

if __name__ == '__main__':
    with Pool(processes=20) as pool:
        res = pool.map(Parser3000.func_wrapper, all_id)

In [16]:
import pandas as pd

df = pd.DataFrame(res)
df.sort_values(by=['ID'], inplace=True)
df = df.fillna(" ")

with open('hw_3.csv', mode='w', encoding='utf-8') as f_csv:
    df.to_csv(f_csv, index=False)

In [17]:
df.head(5)

,ID,ISBN:,Автор:,Авторы:,В базе:,В закладки,Возраст,Год:,Издательство:,Код:,...,Понравилось,Производитель:,Размеры:,Рейтинг,Серия:,Страниц:,Тематика:,Тираж:,Формат:,Цена
952,330658,978-5-699-20894-4,Донцова Дарья Аркадьевна,,Э.ИД.Золушка в шоколаде,-,16+,2007,Эксмо,333104,...,7,,"13,00 см x 20,00 см x 2,50 см",70,Иронический детектив,384,Отечественные,250000,84х108/32,Отсутствует в продаже
1773,330661,978-5-699-20169-3,Емец Дмитрий Александрович,,Э.Мефодий Буслаев-7.Лед и пламя Тартара,26,6+,2013,Эксмо,323903,...,360,,"13,00 см x 20,00 см x 2,60 см",98.1,Мефодий Буслаев,416,Для среднего школьного возраста,90100,205.00mm x 135.00mm x 25.00mm,Отсутствует в продаже
951,330670,5-699-19930-6,Донцова Дарья Аркадьевна,,Э.ИД.Каникулы в Простофилино,-,16+,2007,Эксмо,325231,...,5,,"13,00 см x 20,00 см x 2,50 см",62.5,Иронический детектив,384,Отечественные,250000,84х108/32,Отсутствует в продаже
950,330918,978-5-699-21569-0,Донцова Дарья Аркадьевна,,Э.ИД.Зимнее лето весны,2,16+,2007,Эксмо,343340,...,7,,"13,00 см x 20,00 см x 2,60 см",77.8,Иронический детектив,384,Отечественные,250000,84х108/32,Отсутствует в продаже
949,330920,978-5-699-22452-4,Донцова Дарья Аркадьевна,,Э.ИД.Личное дело Женщины-кошки,-,16+,2007,Эксмо,347336,...,4,,"13,00 см x 20,00 см x 2,70 см",100,Иронический детектив,384,Отечественные,250000,84х108/32,Отсутствует в продаже
